In [1]:
import pandas as pd
import numpy as np
import torch
import pickle

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split

2023-05-19 08:45:48.891793: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 08:45:59.226927: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-19 08:46:20.726861: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /spack/spack-0.17.1/apps/linux-rhel8-cascadelake/gcc-10.2.0/python-3.9.10-y63csltfuw5dh

In [2]:
def overlap_sequence(seq, word_length, overlap):
    if overlap >= word_length:
        print('Overlap must be less than word length')
        return
    
    for i in range(0, len(seq)-overlap, word_length-overlap):
        yield seq[i:i+word_length]
        
def get_overlap_array(seq, word_length=4, overlap=1):
    return np.array(list(overlap_sequence(seq, word_length, overlap)))

def get_overlap_string(seq, word_length=4, overlap=1):
    return ' '.join(list(overlap_sequence(seq, word_length, overlap)))

def compute_metrics(epred):
    # Computes metrics from specialized output from huggingface

    preds = np.exp(epred[0]) / np.sum(np.exp(epred[0]), axis = 0)
    labels = epred[1]

    metrics = {}
    metrics['auprc'] = average_precision_score(labels, preds[:,1])
    metrics['auroc'] = roc_auc_score(labels, preds[:,1])

    return metrics

In [2]:
PATH='/lustre/isaac/proj/UTK0196/deep-surface-protein-data/'
df = pd.read_csv(PATH+'M0059E_training_set.tsv', delimiter=',', header=0)

In [3]:
split_no = 1
inds = pickle.load(open('./splits/splits.pkl', 'rb'))
RUN = 1

In [4]:
train_inds, test_inds = inds[split_no][RUN]

In [6]:
#train_set = df.iloc[train_inds,:]
test_set = df.iloc[test_inds,:]
    
#train_set, val_set = train_test_split(train_set, test_size=0.1, random_state=1234)
    
#train_seqs_list = train_set['surf.sequence'].tolist() + train_set['deep.sequence'].tolist()
#train_seqs_labels = np.concatenate([np.zeros(train_set.shape[0], dtype=int), np.ones(train_set.shape[0], dtype=int)])
    
#val_seqs_list = val_set['surf.sequence'].tolist() + val_set['deep.sequence'].tolist()
#val_seqs_labels = np.concatenate([np.zeros(val_set.shape[0], dtype=int), np.ones(val_set.shape[0], dtype=int)])
    
test_seqs_list = test_set['surf.sequence'].tolist() + test_set['deep.sequence'].tolist()
test_seqs_labels = np.concatenate([np.zeros(test_set.shape[0], dtype=int), np.ones(test_set.shape[0], dtype=int)])
    
#classification_df_train = pd.DataFrame({'text' : train_seqs_list, 'label' : train_seqs_labels})
#classification_df_val = pd.DataFrame({'text' : val_seqs_list, 'label' : val_seqs_labels})
classification_df_test = pd.DataFrame({'text' : test_seqs_list, 'label' : test_seqs_labels})

#classification_df_train['text'] = classification_df_train['text'].transform(get_overlap_string)
#classification_df_val['text'] = classification_df_val['text'].transform(get_overlap_string)
#classification_df_test['text'] = classification_df_test['text'].transform(get_overlap_string)
#med_len = int(np.median([len(elem) for elem in classification_df_test['text']]))

#ds_train = Dataset.from_pandas(classification_df_train)
#ds_val = Dataset.from_pandas(classification_df_val)
ds_test = Dataset.from_pandas(classification_df_test)

In [7]:
tokenizer = AutoTokenizer.from_pretrained('./checkpoint-final')

#tokenized_ds_train = ds_train.map(lambda d : tokenizer(d['text'], truncation=True, padding=True), batched=True)
#tokenized_ds_val = ds_val.map(lambda d : tokenizer(d['text'], truncation=True, padding=True), batched=True)
tokenized_ds_test = ds_test.map(lambda d : tokenizer(d['text'], truncation=True, padding=True), batched=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/92182 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [11]:
model = AutoModelForSequenceClassification.from_pretrained('./models/drbert-test_1/checkpoint-600000', num_labels=2)

In [14]:
trainer = Trainer(
    model=model,
 #   args=training_args,
 #   train_dataset=tokenized_ds_train,
 #   eval_dataset=tokenized_ds_val,
    tokenizer=tokenizer,
)

In [16]:
out = trainer.predict(tokenized_ds_test)
scores = compute_metrics(out)
print(scores)

RuntimeError: The expanded size of the tensor (1129) must match the existing size (1026) at non-singleton dimension 1.  Target sizes: [8, 1129].  Tensor sizes: [1, 1026]